In [1]:
import pickle
import pandas as pd
import nltk
import re

In [2]:
import pandas as pd
import re
import unicodedata

def limpiar_texto(texto):
    
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    texto_limpio = re.sub(r'[^a-zA-Z0-9\s]', '', texto_normalizado)
    return texto_limpio.lower()



In [3]:
puestos = pickle.load(open('static/data/puestos.pickle', 'rb'))
puestos["PUESTO"] = puestos["PUESTO"].apply(limpiar_texto)

In [4]:
### Tokenización
puestos["tokens"] = puestos["PUESTO"].apply(nltk.word_tokenize)

In [5]:
### Stop words

stopwords = nltk.corpus.stopwords.words('spanish')
puestos["tokens_sin_stopwords"] = puestos["tokens"].apply(lambda x: [word for word in x if word not in stopwords])

In [6]:
### Lematización

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

puestos["tokens_lematizados"] = puestos["tokens_sin_stopwords"].apply(lambda x: [wordnet_lemmatizer.lemmatize(word, pos = "v") for word in x])

In [7]:
## Unir
puestos["texto_final"] = puestos["tokens_lematizados"].apply(lambda x: " ".join(x))

In [8]:
### Vectorización

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(puestos["texto_final"])

TfidfVectorizer()

In [9]:
pickle.dump(vectorizer, open('static/data/nuestro_vectorizer.pickle', 'wb'))

In [10]:
pickle.load(open('static/data/nuestro_vectorizer.pickle', 'rb'))

TfidfVectorizer()

In [11]:
import docx

def get_text_from_document(path):
    doc = docx.Document(path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)

    return " ".join(full_text)

def prepare_text(text):
    text = limpiar_texto(text)
    tokens = nltk.word_tokenize(text)
    tokens_sin_stopwords = [word for word in tokens if word not in stopwords]
    tokens_lematizados = [wordnet_lemmatizer.lemmatize(word, pos = "v") for word in tokens_sin_stopwords]
    return " ".join(tokens_lematizados)

def vectorize_text(vect, text):
    return vect.transform(text)

doc = docx.Document("/Users/aladelca/Library/CloudStorage/GoogleDrive-alarcon.adrianc@gmail.com/My Drive/portafolio/cv_matcher/lab2/documents/Carlos Adrián Alarcón Delgado alarcon_prueba.docx")    
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)

full_text = " ".join(full_text)
texto_limpio = limpiar_texto(full_text)
tokens = nltk.word_tokenize(texto_limpio)
tokens_sin_stopwords = [word for word in tokens if word not in stopwords]
tokens_lematizados = [wordnet_lemmatizer.lemmatize(word, pos = "v") for word in tokens_sin_stopwords]
texto_final = " ".join(tokens_lematizados)
texto_final

'carlos adrian alarcon delgado alarconadriancgmailcom 1 514 531 8635 areas inter data science machine learn data analytics data visualization experiencia laboral aldo group montreal canada data scientist may 2024 presente desarrollo modelos predictivos forecast implementacion sistema revenue management impacto directo estrategias price supply chain management universidad peruana ciencias aplicadas lima peru profesor tiempo parcial ago 2023 presente liderar dictado curso data mine data analytics analitica datos sistemas predictivos facultad ingenieria belcorp lima peru data scientist feb 2022 presente desarrollo modelos predictivos diversos proyectos digitales forecast liderar implementacion adopcion dataiku plataforma automl creacion material learn path capacitacion usuarios dataiku data analyst abr 2021 feb 2022 creacion dashboards tableau visual studio liderar estrategia datos proyectos principales activos digitales analisis data interacciones comportamiento digital ecommerce activos

In [12]:
puestos_vectorizados = vectorizer.transform(puestos["texto_final"]) 
documento_vectorizado = vectorizer.transform([texto_final])

In [ ]:
import PyPDF2

# Abre el archivo PDF en modo lectura binaria
def get_pdf(path):
    with open(path, 'rb') as file:
        
        lector = PyPDF2.PdfReader(file)
        final_text = []
        for page_num in range(len(lector.pages)):
            page = lector.pages[page_num]
            text = page.extract_text()
            final_text.append(text)
        return " ".join(final_text)

--- Página 1 ---
M. EXCEL
POWER BI
PYTHON
EXACTUS ERP
SQL SERVER
SAP
Bachiller en Ingeniería Industrial por la
Universidad Peruana de Ciencias Aplicadas y en
Ingeniería Textil y de Confecciones por la
Universidad Nacional Mayor de San Marcos.
Destaco mis habilidades en el Planeamiento de
Operaciones y la gestión efectiva de la cadena
de suministro, así como Inteligencia Comercial
y Planeamiento de la Demanda. 
Resalto mi participación activa en intercambios
estudiantiles en Estados Unidos y Alemania, lo
cual ha enriquecido mi perspectiva y
habilidades interculturales. 
Estoy comprometida con el crecimiento
profesional continuo y la aplicación de mis
conocimientos para contribuir al éxito de los
proyectos y metas empresariales.CENTRUM PUCP Business School2022 - 2022
Programa de Especialización en
Planeamiento y Programación de las
Operaciones
+51 991 757 426
adrianajaureguim@gmail.comAdriana Perez FORMACIÓN PROFESIONAL
INGENERÍA INDUSTRIAL
INGENIERÍA TEXTIL Y  CONFECCIONES
25 AÑOS
CONTA

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

def get_similarity(text_vectorized, jobs, n = 5):
    similarities = cosine_similarity(text_vectorized, jobs)
    top_similarities = pd.Series(similarities[0]).sort_values(ascending = False).head(n)
    indexes = top_similarities.index
    df = pd.DataFrame(jobs.iloc[indexes, 0])
    df["RANKING"] = top_similarities
    df = df.rename(columns = {"PUESTO":"JOBS"})
    df["JOBS"] = df["JOBS"].str.title()
    return df.reset_index(drop = True)

similitudes = cosine_similarity(documento_vectorizado, puestos_vectorizados)
top_similitudes = pd.Series(similitudes[0]).sort_values(ascending = False).head(5)
indices = top_similitudes.index

In [27]:
df = pd.DataFrame(puestos.iloc[indices, 0])
df["RANKING"] = top_similitudes
df["PUESTO"] = df["PUESTO"].str.title()
df.reset_index(drop = True, inplace=True)

In [1]:
### Probar si funciona

from transformer import *

texto = get_text_from_document("/Users/aladelca/Library/CloudStorage/GoogleDrive-alarcon.adrianc@gmail.com/My Drive/portafolio/cv_matcher/lab2/documents/Carlos Adrián Alarcón Delgado alarcon_prueba.docx")
texto = prepare_text(texto)

In [2]:
import pickle
vectorizer = pickle.load(
        open(
            "/Users/aladelca/Library/CloudStorage/GoogleDrive-alarcon.adrianc@gmail.com/My Drive/portafolio/cv_matcher/lab2/matcher/static/data/nuestro_vectorizer.pickle",
            "rb"
        )
    )

jobs = pickle.load(
        open(
            "/Users/aladelca/Library/CloudStorage/GoogleDrive-alarcon.adrianc@gmail.com/My Drive/portafolio/cv_matcher/lab2/matcher/static/data/puestos.pickle",
            "rb"
        )
    )


jobs = prepare_matrix(jobs, "PUESTO")

In [25]:
vect_text = vectorize_text(vectorizer, [texto])
vect_jobs = vectorize_text(vectorizer, jobs["PUESTO"])

In [27]:
get_similarity(vect_text, vect_jobs, jobs, 15)

,JOBS,RANKING
0,Analista Bi Data Analytics Comercial Sql Avanzado,0.440179
1,Analista Bi Data Analytics Comercial Sql Avanzado,0.440179
2,Coordinador Analytics Big Data,0.413254
3,Senior Data Engineer Big Data,0.377279
4,Data Management,0.374018
5,Data Scientist,0.339839
6,Data Scientist,0.339839
7,Gestor Cultura Datos Experiencia Ventas Soluci...,0.332256
8,Data Engineer,0.332150
9,Especialista Data Center,0.326613
